<h2> Load Data</h2>

In [34]:
import warnings
import matplotlib.pyplot as plt
import json
import numpy as np
import pandas as pd

import preparers
import sonaion_analysis as son
import GenSnippetsLib as snippet
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [35]:
data = preparers.load_queried(
    participant_number=1,
    snippets=["BogoSort"],
    query_code=True,
    query_eye_tracking=True,
)

Loading BogoSort Files .....

<h2>potential funtion arguments</h2>

In [36]:
def get_line_bounds(img, image_path):
    height = img.size[1]
    with open(image_path) as json_file:
        data = json.load(json_file)
        height_margin = data["height-margin"]
        loc = len(data["source-code"])
        height = height - 2 * height_margin

    step_size = height/loc
    bounds = []
    for i in range(loc):
        lower = int(i*step_size) + height_margin
        higher = int((i+1)*step_size) + height_margin
        bounds.append((i, lower, higher))

    return bounds

In [37]:
fig, ax = (None, None)
denoise_degree = 10
display_width = 1920
display_height = 1080
left_color = (0.0, 1.0, 1.0)
right_color = (0.0, 1.0, 0.0)
image_path = "../CodeSnippets/Generators/" + "BogoSort" + ".json"
img, result = snippet.create_image(image_path, font_path="/../CodeSnippets/fonts/ttf/")
offset = int(display_height*0.5-img.size[1]*0.5)
img_bounds = get_line_bounds(img, image_path)
img_bounds = [(i, low + offset, high + offset) for i, low, high in img_bounds]
loc = len(img_bounds)
line_middle = [int((low+height)/2) for i, low, height in img_bounds]
bg = snippet.create_background(display_width, display_height, (180, 180, 180, 255))
img = snippet.place_image_on(bg, np.array(img), 0.5, 0.5)
cat = np.full(display_height, None)
for i, low, height in img_bounds:
    for j in range(low, height):
        cat[j] = i

<h2> Prepare queried Data</h2>

In [38]:
def update_names(df_eyetracking, name_dict):
    df_eyetracking = df_eyetracking.rename(columns=name_dict)
    return df_eyetracking

In [39]:
def update_to_display_coordinates(df_eyetracking):
    df_eyetracking["display_x"] = df_eyetracking["display_x"].apply(lambda x: x * display_width)
    df_eyetracking["display_x"] = df_eyetracking["display_x"].astype(float).fillna(0).astype(int)

    df_eyetracking["display_y"] = df_eyetracking["display_y"].apply(lambda x: x * display_height)
    df_eyetracking["display_y"] = df_eyetracking["display_y"].astype(float).fillna(0).astype(int)
    return df_eyetracking

In [40]:
def update_normalize_time(df_eyetracking):
    df_eyetracking["time"] = df_eyetracking["time"] - df_eyetracking["time"][0]
    return df_eyetracking

In [41]:
def update_replace_invalid(df_eyetracking):
    (
        replaced_x,
        replaced_y,
        replaced_diameter,
        replaced_valid,
    ) = son.eyetracking.preprocessing_invalid.replace_with_prev_invalid(
        df_eyetracking["display_x"],
        df_eyetracking["display_y"],
        df_eyetracking["pupil_diameter"],
        df_eyetracking["valid"],
    )

    df_eyetracking["display_x"] = replaced_x
    df_eyetracking["display_y"] = replaced_y
    df_eyetracking["pupil_diameter"] = replaced_diameter
    df_eyetracking["valid"] = replaced_valid
    return df_eyetracking

In [42]:
def update_denoise(df_eyetracking, denoise_degree):
    for _i in range(denoise_degree):
        denoised_x, denoised_y = son.eyetracking.preprocessing_denoise.denoise(
            df_eyetracking["display_x"],
            df_eyetracking["display_y"],
            son.eyetracking.preprocessing_denoise.parabola_5_kernel,
        )

        df_eyetracking["display_x"] = denoised_x
        df_eyetracking["display_y"] = denoised_y
    return df_eyetracking

In [43]:
def update_match_to_line(df_eyetracking, line_middle, cat):
    df_eyetracking["display_y"] =  df_eyetracking["display_y"].apply(lambda x: line_middle[cat[x]] if cat[x] is not None else float("Nan"))
    df_eyetracking = df_eyetracking[df_eyetracking["display_y"].notnull()]
    df_eyetracking["display_y"] = df_eyetracking["display_y"].astype(int)
    df_eyetracking = df_eyetracking.reset_index()
    return df_eyetracking

In [50]:
def mask_function():
    return lambda height, width, coordinate: son.utils.masks.create_circular_mask(height, width, coordinate, 30)

def create_heatmap(df_eyetracking, color, width, height):
    heat_data = son.eyetracking.heatmap.create_heatmap(
        df_eyetracking["display_x"],
        df_eyetracking["display_y"],
        df_eyetracking["valid"],
        0.004,
        width,
        height,
        mask_function(),
    )

    heat_data = heat_data / heat_data.max()
    heat_data = np.array([heat_data * color[0], heat_data * color[1], heat_data * color[2], heat_data])
    heat_data = np.moveaxis(heat_data, 0, 2)
    return heat_data

In [ ]:
def create_sequence_diagram(df_eyetracking, color, width, height, max_pixel):
    step_size = max_pixel/len(df_eyetracking)

    sequence_data = son.eyetracking.sequence.create_sequence_diagram_y(
        df_eyetracking["display_y"],
        df_eyetracking["valid"],
        height,
        width,
        offset=0,
        step=step_size,
        should_skip=False,
    )
    sequence_data = np.array([sequence_data * color[0], sequence_data * color[1], sequence_data * color[2], sequence_data])
    sequence_data = np.moveaxis(sequence_data, 0, 2)
    return sequence_data

In [ ]:
def get_fixation_saccades_metrics(df_eyetracking, d_time):
    d_time = 0.004
    threshold_for_fixation = 2400.0
    saccades = son.eyetracking.metrics.classify_saccades( df_eyetracking["display_x"], df_eyetracking["display_y"], d_time, threshold_for_fixation)
    if len(saccades) >= 1:
        saccades[0] = saccades[1]


    fixxation = son.eyetracking.metrics.classify_fixxation( df_eyetracking["display_x"], df_eyetracking["display_y"], d_time, threshold_for_fixation)
    if len(fixxation) >= 1:
        fixxation[0] = fixxation[1]


    saccades = np.array(saccades)
    fixxation = np.array(fixxation)

    saccades_count = son.eyetracking.metrics.count_saccades(saccades)
    if saccades_count == 0:
        saccades_time_of_first = 0
        saccades_average_time = 0
    else:
        saccades_time_of_first = son.eyetracking.metrics.time_of_saccades(saccades, d_time)[0][0]
        saccades_average_time = son.eyetracking.metrics.average_saccades_time(son.eyetracking.metrics.time_of_saccades(saccades, d_time))

    fixxation_count = son.eyetracking.metrics.count_fixxations(fixxation)
    if fixxation_count == 0:
        fixxation_time_of_first = 0
        fixxation_average_time = 0
    else:
        fixxation_time_of_first = son.eyetracking.metrics.time_of_fixxations(fixxation, d_time)[0][0]
        fixxation_average_time = son.eyetracking.metrics.average_fixxations_time(son.eyetracking.metrics.time_of_fixxations(fixxation, d_time))

    return pd.DataFrame([[
        saccades_count,
        saccades_time_of_first,
        saccades_average_time,
        fixxation_count,
        fixxation_time_of_first,
        fixxation_average_time,
    ]], columns=["Saccades Count", "Time to first Saccade", "Average Saccade Time", "Fixation Count", "Time to first Fixation", "Average Fixation Time"])

In [47]:
df_eyetracking = data["BogoSort"]["Code"]["EyeTracking"][
    [
        "l_valid",
        "l_display_x",
        "l_display_y",
        "time",
        "l_pupil_diameter",
    ]
]

left_naming = {
    "l_valid" : "valid",
    "l_display_x" : "display_x",
    "l_display_y" : "display_y",
    "l_pupil_diameter" : "pupil_diameter",
}

right_naming = {
    "r_valid" : "valid",
    "r_display_x" : "display_x",
    "r_display_y" : "display_y",
    "r_pupil_diameter" : "pupil_diameter",
}

df_eyetracking = update_names(df_eyetracking, left_naming)
df_eyetracking = update_to_display_coordinates(df_eyetracking)
df_eyetracking = update_normalize_time(df_eyetracking)
df_eyetracking = update_replace_invalid(df_eyetracking)
df_eyetracking = update_denoise(df_eyetracking, denoise_degree)
df_eyetracking = update_match_to_line(df_eyetracking, line_middle, cat)